# Les décorateurs et les différentes méthodes 

## 1. Introduction aux décorateurs

### a. Décors de fonctions:

Un décorateur est essentiellement une fonction qui en "décore" ou en modifie une autre. Les décorateurs sont une manière élégante de modifier le comportement d'une fonction sans la changer directement. En Python, les décorateurs sont appliqués à l'aide du symbole **@** suivi du nom du décorateur.

#### Comment fonctionne un décorateur ?
Prenons un décorateur simple comme exemple :

In [12]:
def simple_decorator(func):
    def wrapper():
        print("Avant l'appel de la fonction.")
        func()
        print("Après l'appel de la fonction.")
    return wrapper

@simple_decorator
def say_hello():
    print("Hello!")


In [13]:
say_hello()

Avant l'appel de la fonction.
Hello!
Après l'appel de la fonction.


Lorsque vous déclarez **@simple_decorator** au-dessus de say_hello, cela équivaut à dire :

In [10]:
say_hello = simple_decorator(say_hello)

#### Ce qui se passe :

**func** : Lorsque vous utilisez un décorateur, la fonction que vous "décorez" est passée au décorateur comme argument. Dans cet exemple, func est une référence à say_hello.

**wrapper** : Il s'agit d'une fonction interne (ou de fermeture) qui englobe la fonction originale. Elle permet d'exécuter du code avant et/ou après l'appel de la fonction originale.

**func()** dans le wrapper appelle la fonction originale.

Le décorateur retourne le wrapper.

Lorsque vous appelez **say_hello()**, vous appelez en réalité le wrapper qui appelle **say_hello** à l'intérieur.

#### Mot-clés ou convention ?

Les mots "wrapper" et "func" ne sont que des conventions et ne sont en aucun cas des obligations strictes. Vous êtes libre de choisir n'importe quel autre nom pour ces fonctions internes ou pour l'argument de la fonction décoratrice. Cependant, il est courant d'utiliser des noms comme "wrapper" ou "inner" pour la fonction interne et des noms comme "func", "f", ou "function" pour l'argument de la fonction décoratrice, car ils décrivent clairement leurs rôles respectifs.

Voici un exemple pour illustrer ce point :

In [5]:
def my_decorator(original_function):
    def inner_function():
        print("Avant l'appel de la fonction.")
        original_function()
        print("Après l'appel de la fonction.")
    return inner_function

@my_decorator
def greet():
    print("Hello!")

Dans cet exemple, "my_decorator" est la fonction décoratrice, "original_function" est l'argument de la fonction décoratrice (la fonction que vous décorez), et "inner_function" est la fonction interne qui enveloppe et modifie le comportement de "original_function".

#### Exemple avec arguments:

Les décorateurs peuvent également être utilisés avec des fonctions qui prennent des arguments :

In [14]:
def decorator_with_args(func):
    def wrapper(*args, **kwargs):
        print(f"Arguments : {args}, {kwargs}")
        return func(*args, **kwargs)
    return wrapper

@decorator_with_args
def greet(name, age=None):
    if age:
        return f"Hello {name}, you are {age} years old!"
    else:
        return f"Hello {name}!"

In [15]:
greet("Camille", age=31)

Arguments : ('Camille',), {'age': 31}


'Hello Camille, you are 31 years old!'

#### Dans cet exemple :

*args et **kwargs dans le wrapper permettent de passer n'importe quel nombre d'arguments positionnels et nommés à la fonction originale.
Exercice sur les décorateurs de fonctions:

Créez un décorateur nommé execution_time qui mesure et affiche le temps qu'il a fallu pour exécuter la fonction décorée.

### b. Décorateurs de méthodes :
Les méthodes sont, dans l'essence, des fonctions qui sont définies à l'intérieur d'une classe et opèrent sur des instances de cette classe. Par conséquent, vous pouvez également les décorer comme des fonctions. Cependant, il y a une subtilité importante à prendre en compte.

Lorsque vous décorez une méthode, le premier argument qu'elle reçoit est toujours une référence à l'instance elle-même (habituellement self). Si c'est une méthode de classe, alors elle reçoit la classe elle-même comme premier argument (habituellement cls).

Voici comment vous pouvez décorer une méthode d'instance :

In [1]:
def method_decorator(func):
    def wrapper(self, *args, **kwargs):
        print(f"Avant d'appeler {func.__name__}")
        result = func(self, *args, **kwargs)
        print(f"Après avoir appelé {func.__name__}")
        return result
    return wrapper

class MaClasse:
    def __init__(self, value):
        self.value = value

    @method_decorator
    def ma_methode(self, param):
        print(f"Méthode appelée avec {param}. Valeur interne: {self.value}")

instance = MaClasse(42)
instance.ma_methode("test")


Avant d'appeler ma_methode
Méthode appelée avec test. Valeur interne: 42
Après avoir appelé ma_methode


Dans l'exemple ci-dessus, method_decorator est utilisé pour décorer ma_methode. Lorsque vous appelez cette méthode, les messages "Avant d'appeler" et "Après avoir appelé" sont affichés avant et après l'appel de la méthode réelle.

Notez que dans la fonction wrapper du décorateur, le premier argument est self. C'est la référence à l'instance sur laquelle la méthode est appelé

## 2. Les différents types de méthodes
### a. Méthodes d'instance
#### Définition
Ce sont les méthodes traditionnelles que nous associons à un objet. Elles ont accès aux données de l'objet via self.

#### Exemple
Supposons qu'une bibliothèque ait des adhérents et chaque adhérent peut emprunter des livres.

In [2]:
class Adherent:
    def __init__(self, nom, prenom):
        self.nom = nom
        self.prenom = prenom
        self.livres_empruntes = []

    def emprunter(self, livre):
        self.livres_empruntes.append(livre)
        print(f"{self.prenom} {self.nom} a emprunté {livre.titre}.")

    def retourner(self, livre):
        if livre in self.livres_empruntes:
            self.livres_empruntes.remove(livre)
            print(f"{self.prenom} {self.nom} a retourné {livre.titre}.")


### b. Méthodes de classe (classmethod)

#### Définition

Elles sont liées à la classe et non à une instance de la classe. Elles ont accès aux attributs de classe, mais pas aux attributs d'instance. Elles sont généralement utilisées pour créer des constructeurs alternatifs. En Python, on utilise le décorateur @classmethod.

#### Exemple
Disons que chaque livre a un identifiant unique et une méthode pour incrémenter cet identifiant à chaque fois qu'un nouveau livre est ajouté.

In [8]:
class Livre:
    _id_livre = 0

    def __init__(self, titre, auteur):
        self.titre = titre
        self.auteur = auteur
        self.id = Livre._id_livre
        Livre.incr_id()

    @classmethod
    def incr_id(cls):
        cls._id_livre += 1

Ici, nous venons de découvrir une nouvelle fonctionnalité. 

Il est possible de définir une variable en dehors d'une méthode, directement dans les classes : les variables de classe.
Les variables de classe sont définies en dehors de toute méthode, directement à l'intérieur de la classe. Elles ne sont pas liées à une instance particulière de la classe, mais sont partagées entre toutes les instances. Par conséquent, elles sont souvent utilisées pour des propriétés ou des constantes qui sont communes à toutes les instances de la classe.Dans l'exemple donné, _id_livre est une variable de classe. Elle est destinée à être utilisée comme un compteur qui s'incrémente à chaque fois qu'un nouveau livre est créé. Cela peut être utilisé pour donner un identifiant unique à chaque livre. Chaque fois que nous créons un nouveau livre, la variable de classe _id_livre s'incrémente, garantissant que chaque livre a un id unique.

In [9]:
livre1 = Livre("Anna Karenina", "Leo Tolstoy")
print(livre1.id)  # Affiche : 0

livre2 = Livre("War and Peace", "Leo Tolstoy")
print(livre2.id)  # Affiche : 1


0
1


Remarques:

- Les variables de classe peuvent être accessibles et modifiées via le nom de la classe (comme Livre._id_livre dans l'exemple ci-dessus) ou via une instance (comme livre1._id_livre), mais il est généralement recommandé d'y accéder via le nom de la classe pour clarifier que c'est une propriété de la classe et non d'une instance spécifique.

- Il est courant d'utiliser un underscore _ avant le nom de la variable pour indiquer qu'elle doit être traitée comme "privée" et ne pas être modifiée directement en dehors de la classe.

In [17]:
# Définissons d'abord la classe Livre
class Livre:
    _id_livre = 0  # Variable de classe

    def __init__(self, titre, auteur):
        self.titre = titre
        self.auteur = auteur
        self.id = Livre._id_livre
        Livre._id_livre += 1

# Créons une instance de Livre
livre1 = Livre("Anna Karenina", "Leo Tolstoy")

# La variable de classe peut être accédée à la fois par l'instance et par la classe elle-même
print("Après la création du premier livre:")
print("livre1._id_livre:", livre1._id_livre)  # Affiche : 1
print("Livre._id_livre:", Livre._id_livre)   # Affiche : 1

# Modifions la variable de classe via l'instance
livre1._id_livre = 10

# Vérifions les valeurs après modification
print("\nAprès avoir modifié _id_livre à travers l'instance livre1:")
print("livre1._id_livre:", livre1._id_livre)  # Affiche : 10
print("Livre._id_livre:", Livre._id_livre)   # Affiche : 1 (n'a pas été modifié)

# Créons une autre instance de Livre
livre2 = Livre("War and Peace", "Leo Tolstoy")

# Vérifions les valeurs
print("\nAprès la création du second livre:")
print("livre2._id_livre:", livre2._id_livre)  # Affiche : 2 (car la valeur a été incrémentée lors de la création de l'instance)
print("Livre._id_livre:", Livre._id_livre)   # Affiche : 2

# Modifions la variable de classe via la classe elle-même
Livre._id_livre = 20

# Vérifions les valeurs après modification
print("\nAprès avoir modifié _id_livre à travers la classe Livre:")
print("livre2._id_livre:", livre2._id_livre)  # Affiche : 2 (la valeur de l'instance ne change pas)
print("Livre._id_livre:", Livre._id_livre)   # Affiche : 20


# Créons une autre instance de Livre
livre3 = Livre("L'étranger", "Alberet Camus")

# Vérifions les valeurs
print("\nAprès la création du troisieme livre:")
print("livre3._id_livre:", livre2._id_livre)  # Affiche : 21 (car la valeur a été incrémentée lors de la création de l'instance depuis la base 20)
print("Livre._id_livre:", Livre._id_livre)   # Affiche : 21

Après la création du premier livre:
livre1._id_livre: 1
Livre._id_livre: 1

Après avoir modifié _id_livre à travers l'instance livre1:
livre1._id_livre: 10
Livre._id_livre: 1

Après la création du second livre:
livre2._id_livre: 2
Livre._id_livre: 2

Après avoir modifié _id_livre à travers la classe Livre:
livre2._id_livre: 20
Livre._id_livre: 20

Après la création du troisieme livre:
livre3._id_livre: 21
Livre._id_livre: 21


Ce que vous pouvez observer ici :

Quand vous modifiez _id_livre via une instance (comme livre1._id_livre = 10), vous créez en réalité un nouvel attribut d'instance pour livre1 qui "masque" la variable de classe. La variable de classe _id_livre reste inchangée.

C'est pourquoi, quand vous créez un nouvel objet (livre2), le compteur s'incrémente sur la base de la variable de classe et non sur la valeur modifiée de livre1.

Si vous modifiez la variable de classe via la classe elle-même (comme Livre._id_livre = 20), cela affectera tous les futurs objets créés. Cependant, les attributs d'instance existants ne seront pas modifiés.

## Méthodes statiques (staticmethod)

### Définition
Ces méthodes n'ont accès ni aux attributs d'instance (self), ni aux attributs de classe (cls). Elles se comportent comme des fonctions normales, sauf qu'elles appartiennent à la classe de l'objet. Elles sont définies à l'aide du décorateur @staticmethod.

### Exemple
Supposons que dans une bibliothèque, il y a des règles standard pour catégoriser les livres en fonction de leur titre. Par exemple, tous les titres commençant par "A" à "M" sont stockés dans la section 1, tandis que ceux commençant par "N" à "Z" sont stockés dans la section 2. Cette règle est indépendante d'un livre spécifique, c'est plutôt une règle générale que la bibliothèque suit. Une telle règle peut être représentée comme une méthode statique.


In [6]:
class Livre:
    def __init__(self, titre, auteur):
        self.titre = titre
        self.auteur = auteur
        self.section = Livre.definir_section(titre)
        # ... (autres attributs)
    
    @staticmethod
    def definir_section(titre):
        if "A" <= titre[0] <= "M":
            return "Section 1"
        elif "N" <= titre[0] <= "Z":
            return "Section 2"
        else:
            return "Section Autre"

Avec cet exemple, lorsque nous créons un nouveau Livre, la méthode statique definir_section est appelée avec le titre du livre comme argument pour déterminer sa section. Cette méthode est statique parce qu'elle ne dépend d'aucun attribut ou comportement spécifique à une instance particulière de Livre, mais elle est plutôt basée sur une règle générale de la bibliothèque.

Voici comment cela pourrait être utilisé :

In [7]:
livre1 = Livre("Anna Karenina", "Leo Tolstoy")
print(livre1.section)  # Affiche : Section 1

livre2 = Livre("War and Peace", "Leo Tolstoy")
print(livre2.section)  # Affiche : Section 2

Section 1
Section 2


---

### Exercice : Gestion d'une chaîne de restaurants
Imaginez que vous gérez une chaîne de restaurants et souhaitez avoir une application pour suivre le nombre de repas vendus dans chaque restaurant et obtenir quelques statistiques.

1. Créez une classe Restaurant qui a les attributs suivants :
- nom (Nom du restaurant)
- ville (Ville où il est situé)
- repas_vendus (nombre de repas vendus, initialement à 0)
  
2. La classe doit également avoir une variable de classe appelée total_repas_vendus pour suivre le nombre total de repas vendus dans tous les restaurants de la chaîne.

3. La classe doit avoir les méthodes suivantes :

- vendre_repas(self, nombre) : Cette méthode augmentera repas_vendus par le nombre donné et augmentera également la variable de classe total_repas_vendus.
- statistiques_restaurant(self) : Cette méthode affichera les statistiques du restaurant, telles que son nom, sa ville et le nombre de repas vendus.
  
4. Créez une méthode de classe statistiques_chaine qui affiche le nombre total de repas vendus dans toute la chaîne de restaurants.

5. Créez une méthode statique evaluer_repas qui prend une note (de 1 à 5) et retourne une appréciation ("Mauvais", "Moyen", "Bon", "Très bon", "Excellent").

Instructions :

1. Créez les instances de restaurants suivantes :
- restaurant1 avec le nom "Le Gourmet", situé à "Paris".
- restaurant2 avec le nom "Pasta Delight", situé à "Rome".
- restaurant3 avec le nom "Burger Queen", situé à "New York".
2. Vendez 50 repas dans restaurant1, 30 repas dans restaurant2, et 60 repas dans restaurant3 en utilisant la méthode vendre_repas.
3. Affichez les statistiques de chaque restaurant individuellement.
4. Affichez les statistiques globales de la chaîne de restaurants.
5. Évaluez un repas en donnant une note de 4 en utilisant la méthode statique evaluer_repas.

---

## Aller plus loin avec les classes 

Voici une liste des concepts et fonctionnalités orientés objet en Python que nous n'avons pas encore abordé en détail :

**Variables privées** : En Python, il n'y a pas de moyen strict pour rendre une variable ou une méthode totalement privée. Cependant, en ajoutant deux underscores devant le nom (comme __ma_variable), vous pouvez la rendre "privée" dans le sens où elle sera moins accessible de l'extérieur. Ceci est appelé "mangling" et le but est de rendre accidentellement difficile la surcharge ou l'accès à cette variable ou méthode depuis une sous-classe ou depuis l'extérieur de la classe.

**Getters et Setters :**

Getters : Ce sont des méthodes qui permettent d'accéder à la valeur d'une variable privée.
Setters : Ce sont des méthodes qui permettent de modifier la valeur d'une variable privée.
En Python, les getters et setters sont souvent utilisés en conjonction avec la propriété property.
**Les property :** C'est une manière pythonique de définir des getters et setters. Avec property, vous pouvez définir une méthode comme une propriété d'une classe, et lorsqu'elle est appelée, elle agit comme si vous accédiez à un attribut plutôt qu'à une méthode.

**Les méthodes du "dunder" :** Outre __init__, __str__, et __repr__ que nous avons déjà abordées, il existe de nombreuses autres méthodes spéciales, comme __add__, __eq__, __len__, etc., qui vous permettent de définir des comportements personnalisés pour des opérations courantes.

**Méthodes abstraites et classes abstraites :** Une classe abstraite est une classe qui ne peut pas être instanciée et est conçue pour être sous-classée. Les méthodes abstraites dans une classe abstraite doivent être implémentées par toute sous-classe.

**Composition et Agrégation :** Ce sont des alternatives à l'héritage qui permettent de construire des classes en utilisant d'autres classes comme composants plutôt que de les sous-classer.

**Mixins :** Ce sont des petites classes qui fournissent un ensemble spécifique de fonctionnalités supplémentaires que vous pouvez utiliser pour étendre d'autres classes.

**Les décorateurs de classe :** Tout comme il existe des décorateurs de fonction, vous pouvez également définir des décorateurs pour les classes afin d'étendre ou de modifier leur comportement.

**Les métaclasses :** Ce sont des classes "d'un niveau supérieur" qui définissent le comportement des classes elles-mêmes. C'est un concept avancé et souvent non nécessaire pour la plupart des utilisations courantes.

**Encapsulation, Polymorphisme, et Héritage :** Bien que nous ayons déjà abordé ces sujets, il est toujours possible de les explorer plus en profondeur, en particulier leurs applications pratiques et leurs nuances.